In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/miniconda3/envs/nca-alife-torch/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [3]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [35]:
header = '\n'.join([
    '#!/bin/bash',
    
    # 'source ~/.virtualenvs/nca-alife-torch/bin/activate'
    
    'source ~/miniconda3/etc/profile.d/conda.sh',
    'conda activate nca-alife-torch',
    
    'cd ~/nca-alife/src'
])


gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 48 commands: 
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_scale4/0_a very fat cat_4"                  --dtype="float32" --device="cuda:0" --img_size=4   --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="point" --padding_mode="circular" --dt=0.01 --dropout=0.1 --prompts="a very fat cat"                --spatial_scales="1." --clip_model="clip-vit-base-patch32" --coef_alignment=1.0 --coef_spatial_softmax=0.0 --rollout_steps=128 --bptt_steps=16 --pool_size=128 --bs=8 --lr=0.0003 --n_iters=100000 --clip_grad_norm=1.0
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_scale4/0_a biological cell_4"               --dtype="float32" --device="cuda:0" --img_size=4   --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="point" --padding_mode="circular" --dt=0.01 --dropout=0.1 --prompts="a biological cell"             --spatial_scales

In [36]:
print(log_dir)

/home/akarshkumar0101/nca-alife-data/logs/2024_07_31_05_26_53


In [37]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [38]:
print('\n'.join(txt.split('\n')[:24]))

#!/bin/bash
source ~/miniconda3/etc/profile.d/conda.sh
conda activate nca-alife-torch
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_05_26_53/job_00003.status
python train_nca_temporal_emergence.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_scale4/0_particles in a gas_4"              --dtype="float32" --device="cuda:0" --img_size=4   --d_state=16 --perception="fullconv" --kernel_size=3 --init_state="point" --padding_mode="circular" --dt=0.01 --dropout=0.1 --prompts="particles in a gas"            --spatial_scales="1." --clip_model="clip-vit-base-patch32" --coef_alignment=1.0 --coef_spatial_softmax=0.0 --rollout_steps=128 --bptt_steps=16 --pool_size=128 --bs=8 --lr=0.0003 --n_iters=100000 --clip_grad_norm=1.0 &> /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_05_26_53/job_00003.out
echo $? > /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_05_26_53/job_00003.status

touch /home/akarshkumar0101/nca-

In [39]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_05_26_53/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_05_26_53/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_05_26_53/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_05_26_53/gpu_7.sh


# Track Experiment

In [22]:
import subprocess
a = subprocess.run(['ps', 'ux'], stdout=subprocess.PIPE).stdout.decode('utf-8')

In [25]:
[l for l in a.split('\n') if "bash" in l and "gpu" in l]

['akarshk+ 1271851  0.0  0.0   2480   572 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_4.sh',
 'akarshk+ 1271852  0.0  0.0   2480   516 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_5.sh',
 'akarshk+ 1271853  0.0  0.0 219972  3408 ?        S    04:58   0:00 bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_4.sh',
 'akarshk+ 1271854  0.0  0.0   2480   512 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_6.sh',
 'akarshk+ 1271855  0.0  0.0 219972  3200 ?        S    04:58   0:00 bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_5.sh',
 'akarshk+ 1271856  0.0  0.0   2480   516 ?        S    04:58   0:00 /bin/sh -c bash /home/akarshkumar0101/nca-alife-data/logs/2024_07_31_04_58_39/gpu_7.sh',
 'akarshk+ 1271857  0.0  0.0 219972  3460 ?        S    04:58   0:00 bash 

Terminated
Terminated
Terminated
Terminated


In [40]:
from collections import defaultdict

statuses = []
for job_id in range(len(commands)):
    try:
        with open(f"{log_dir}/job_{job_id:05d}.status", "r") as f:
            status = int(f.read().strip())
            statuses.append(status)
    except Exception as e:
        pass
        # print(e)
        # break
statuses = np.array(statuses)

In [41]:
(statuses==0).sum(), len(commands)

(0, 48)

In [42]:
np.unique(statuses, return_counts=True)

(array([], dtype=float64), array([], dtype=int64))

In [55]:
for job_id in list(range(len(commands))):
    print()
    print(f'------------------------------------------------------ {job_id} ------------------------------------------------------')
    print()
    try:
        with open(f"{log_dir}/job_{job_id:05d}.out", "r") as f:
            print(f.readlines()[-1].strip())
    except Exception as e:
        # pass
        break
        print(e)


------------------------------------------------------ 0 ------------------------------------------------------

100%|██████████| 100000/100000 [49:04<00:00, 33.96it/s, loss=-0.282]

------------------------------------------------------ 1 ------------------------------------------------------

100%|██████████| 100000/100000 [50:06<00:00, 33.26it/s, loss=-0.304]

------------------------------------------------------ 2 ------------------------------------------------------

100%|██████████| 100000/100000 [50:49<00:00, 32.80it/s, loss=-0.251]

------------------------------------------------------ 3 ------------------------------------------------------

100%|██████████| 100000/100000 [51:17<00:00, 32.49it/s, loss=-0.319]

------------------------------------------------------ 4 ------------------------------------------------------

100%|██████████| 100000/100000 [49:26<00:00, 33.71it/s, loss=-0.258]

------------------------------------------------------ 5 ---------------------------

In [54]:
log_dir

'/home/akarshkumar0101/nca-alife-data/logs/2024_07_31_05_26_53'